## Scalability Testing

This notebook is for testing purposes.

In [1]:
from simulator import Simulator
from simulator.config import DATA_PATH, Color
from simulator.helpers import clean
from simulator.helpers.coordinates import ENUPose, GRAPose
from simulator.planner import AutoPlan, Plan
from simulator.visualizer import NoVisualizer, SimVehicle

clean()

## Simulation Positions

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs_colors=[Color.RED,Color.ORANGE,Color.GREEN,Color.BLUE]
n_uavs_per_gcs = 60
side_len = 10
altitude = 5

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs_colors))
    for j in range(n_uavs_per_gcs)
])

## Create Vehicles

In [3]:
sysids = range(1, len(base_homes)+1)
colors: list[Color]= []
for color in gcs_colors:
    colors.extend([color]*n_uavs_per_gcs)

vehs:list[SimVehicle] = []
for sysid, base_home, color in zip(sysids, base_homes, colors):
    auto_plan = AutoPlan.square_traj(
        side_len=side_len,
        alt=altitude,
        name="simple_auto_plan",
        sysid=sysid,
        gra_origin=gra_origin,
        relative_home=base_home,
    )

    veh = SimVehicle.from_relative(
        sysid=sysid,
        gcs_name=f'{color.name}_{color.emoji}',
        plan=auto_plan,
        color=color,
        enu_origin=enu_origin,
        relative_home=base_home,
        relative_path=Plan.create_square_path(side_len=side_len),
        model="iris",
    )
    vehs.append(veh)
    

## Simulator

In [4]:
simulator = Simulator(
	visualizer=NoVisualizer(gra_origin),
	terminals=['gcs'],
	verbose=1,
)
for veh in vehs:
    simulator.add_vehicle(veh)

## Run

In [ ]:
orac = simulator.launch()
orac.run()

05:43:09 - Oracle ⚪ - INFO - 🙈 Running without visualization.
05:43:09 - Oracle ⚪ - INFO - 🚀 GCS RED_🟥 launched (PID 336177)
05:43:09 - Oracle ⚪ - INFO - 🚀 GCS ORANGE_🟧 launched (PID 336178)
05:43:09 - Oracle ⚪ - INFO - 🚀 GCS GREEN_🟩 launched (PID 336179)
05:43:09 - Oracle ⚪ - INFO - 🚀 GCS BLUE_🟦 launched (PID 336180)
05:43:09 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 240 vehicles and 4 GCSs
